# Importando Bibliotecas

In [ ]:
import numpy as np
import CoolProp.CoolProp as CP

# Análise Energética

## Constantes

In [ ]:
M = 28.97 # kg/kmol massa molar do ar
R_bar = 8314 # J/(kmol K)
profit = 0.115 # $US / (kW h)


## Condições conhecidas

### Entrada de produtos de combustão 

In [ ]:
p1 = 1 # atm
t1 = 400 # Fahrenheit
av1 = 2e5 # ft³/min

# Convertendo para SI
p1 = p1 * 1.01325e5 # Pa | N/m²

t1 =  ((t1 - 32) / 1.8) + 273.15 # kelvin
av1 = (2e5 * 0.028317) / 60 # m³/s


### Sainda do da combustão

In [ ]:
p2 = 1 # atm
t2 = 260 # Fahrenheit

# Convertendo para SI
p2 = p2 * 1.01325e5 # Pa | N/m²
t2 =  ((t2 - 32) / 1.8) + 273.15 # kelvin


### Entrada da água

In [ ]:
p3 = 40 # lbf/in²
t3 = 102 # Fahrenheit
m3_dot = 275 # lb/min

# Convertendo para SI
p3 = p3 * 6894.8 # Pa | N/m²
t3 =  ((t3 - 32) / 1.8) + 273.15 # Kelvin
m3_dot = (m3_dot * 0.4536) / 60 # kg/s

### Saida da água

In [ ]:
p5 = 1 # lbf/in²
x5 = 93 # % qualidade

# Convertendo para SI
p5 = p5 * 6894.8  # Pa | N/m²
x5 = x5 / 100 # admensional

## Taxa de fluxo de massa e propriedades

#### Calculando 
$$\dot{m_1} = \frac{(AV)_1}{\upsilon_1}$$
$$ \upsilon_1 = \frac{\bar{R}T_1}{Mp_1} $$


In [ ]:
u1 = (R_bar * t1) / (M * p1) # m³/kg
m1_dot = av1 / u1 # kg/s


### Adquirindo as entalpias pelo CoolProp

In [ ]:
h1 = CP.PropsSI('H', 'T', t1, 'P', p1, 'Air')  # J/kg
h2 = CP.PropsSI('H', 'T', t2, 'P', p2, 'Air')  # J/kg
h3 = CP.PropsSI('H', 'T', t3, 'P', p3, 'Water') # J/kg
h5 = CP.PropsSI('H', 'P', p5, 'Q', x5, 'Water') # J/kg


### Calculando a portencia da turbina 

$$ \dot{W}_{cv} = \dot{m}_1 \left ( h_1 - h_2  \right ) + \dot{m}_3 \left ( h_3 - h_5 \right ) $$

In [ ]:
w_dot = m1_dot * (h1-h2) + m3_dot * (h3 - h5) # w

f"Potencia da turbina = {w_dot * 1e-3} kW"


### Calculando temperatura antes de passar pela turbina (efetuar trabalho)

$$ 0 = \dot{m}_1 \left ( h_1 - h_2  \right ) + \dot{m}_3 \left ( h_3 - h_4 \right ) $$

$$ h_4 = \frac{\dot{m}_1}{\dot{m}_3} \left ( h_1 - h_2  \right ) + h_3  $$

In [ ]:
h4 = (m1_dot / m3_dot ) * (h1 - h2) + h3  # Balanço de energia

t4 = CP.PropsSI('T', 'P', p3 , 'H', h4, 'Water')  # K

f"Temperatura T4 = {t4} K"

### Ganho anual $(8000)$ hours

In [ ]:
VA = w_dot * 1e-3 * 8000 * profit

f"Ganho Anual: $US {VA:.2f}"